# 导入库函数

In [1]:
from __future__ import division
from __future__ import print_function
from gravity_gae.evaluation import compute_scores
from gravity_gae.input_data import load_data
from gravity_gae.model import *
from gravity_gae.optimizer import OptimizerAE, OptimizerVAE
from gravity_gae.preprocessing import *
import numpy as np
import os
import scipy.sparse as sp
import tensorflow as tf
import time

tf.app.flags.DEFINE_string('f', '', 'kernel')#添加的，不报错

# 定义超参数

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.logging.set_verbosity(tf.logging.ERROR)

flags = tf.app.flags
FLAGS = flags.FLAGS

flags.DEFINE_string('dataset', 'Wisconsin', 'Name of the graph dataset')

# Select machine learning task to perform on graph
flags.DEFINE_string('task', 'task_2', 'Name of the link prediction task')
''' See section 4.1. of paper for details about tasks:

- task_1: General Directed Link Prediction

- task_2: Biased Negative Samples Directed Link Prediction

- task_3: Bidirectionality Prediction
'''

# Model
flags.DEFINE_string('model', 'source_target_gcn_vae', 'Name of the model')
''' Available Models:

- gcn_ae: Graph Autoencoder from Kipf and Welling (2016), with 2-layer
          GCN encoder and inner product decoder

- gcn_vae: Variational Graph Autoencoder from Kipf and Welling (2016), with
           Gaussian priors, 2-layer GCN encoders and inner product decoder

- source_target_gcn_ae: Source-Target Graph Autoencoder, as introduced
                        in section 2.6 of paper, with 2-layer GCN encoder
                        and asymmetric inner product decoder

- source_target_gcn_vae: Source-Target Graph Variational Autoencoder, as
                         introduced in section 2.6, with Gaussian priors,
                         2-layer GCN encoders and asymmetric inner product
 
- gravity_gcn_ae: Gravity-Inspired Graph Autoencoder, as introduced in
                  section 3.3 of paper, with 2-layer GCN encoder and 
                  gravity-inspired asymmetric decoder
 
- gravity_gcn_vae: Gravity-Inspired Graph Variational Autoencoder, as
                   introduced in section 3.4 of paper, with Gaussian 
                   priors, 2-layer GCN encoders and gravity-inspired decoder
'''

# Model parameters
flags.DEFINE_float('dropout', 0., 'Dropout rate (1 - keep probability).')
flags.DEFINE_integer('epochs', 200, 'Number of epochs in training.')
flags.DEFINE_boolean('features', False, 'Include node features or not in GCN')
flags.DEFINE_float('lamb', 1., 'lambda parameter from Gravity AE/VAE models \
                                as introduced in section 3.5 of paper, to \
                                balance mass and proximity terms')
flags.DEFINE_float('learning_rate', 0.1, 'Initial learning rate (with Adam)')
flags.DEFINE_integer('hidden', 64, 'Number of units in GCN hidden layer.')
flags.DEFINE_integer('dimension', 32, 'Dimension of GCN output: \
- equal to embedding dimension for standard AE/VAE and source-target AE/VAE \
- equal to (embedding dimension - 1) for gravity-inspired AE/VAE, as the \
last dimension captures the "mass" parameter tilde{m}')
flags.DEFINE_boolean('normalize', False, 'Whether to normalize embedding \
                                          vectors of gravity models')
flags.DEFINE_float('epsilon', 0.01, 'Add epsilon to distances computations \
                                       in gravity models, for numerical \
                                       stability')
# Experimental setup parameters
flags.DEFINE_integer('nb_run', 10, 'Number of model run + test')
flags.DEFINE_float('prop_val', 5., 'Proportion of edges in validation set \
                                   (for Task 1)')
flags.DEFINE_float('prop_test', 10., 'Proportion of edges in test set \
                                      (for Tasks 1 and 2)')
flags.DEFINE_boolean('validation', False, 'Whether to report validation \
                                           results  at each epoch (for \
                                           Task 1)')
flags.DEFINE_boolean('verbose', True, 'Whether to print comments details.')

In [3]:
# Lists to collect average results
mean_roc = []
mean_ap = []
mean_time = []


# Load graph dataset
if FLAGS.verbose:
    print("Loading data...")
adj_init, features = load_data(FLAGS.dataset)


# The entire training process is repeated FLAGS.nb_run times
for i in range(FLAGS.nb_run):

    # Edge Masking: compute Train/Validation/Test set
    if FLAGS.verbose:
        print("Masking test edges...")

    if FLAGS.task == 'task_1':
        # Edge masking for General Directed Link Prediction
        adj, val_edges, val_edges_false, test_edges, test_edges_false = \
        mask_test_edges_general_link_prediction(adj_init, FLAGS.prop_test,
                                                FLAGS.prop_val)
    elif FLAGS.task == 'task_2':
        # Edge masking for B.N.S. Directed Link Prediction
        adj, val_edges, val_edges_false, test_edges, test_edges_false = \
        mask_test_edges_biased_negative_samples(adj_init, FLAGS.prop_test)
    elif FLAGS.task == 'task_3':
        # Edge masking for Bidirectionality Prediction
        adj, val_edges, val_edges_false, test_edges, test_edges_false = \
        mask_test_edges_bidirectional_link_prediction(adj_init)
    else:
        raise ValueError('Undefined task!')

    # Preprocessing and initialization
    if FLAGS.verbose:
        print("Preprocessing and Initializing...")
    # Compute number of nodes
    num_nodes = adj.shape[0]
    # If features are not used, replace feature matrix by identity matrix
    if not FLAGS.features:
        features = sp.identity(adj.shape[0])
    # Preprocessing on node features
    features = sparse_to_tuple(features)
    num_features = features[2][1]
    features_nonzero = features[1].shape[0]

    # Define placeholders
    placeholders = {
        'features': tf.sparse_placeholder(tf.float32),
        'adj': tf.sparse_placeholder(tf.float32),
        'adj_orig': tf.sparse_placeholder(tf.float32),
        'dropout': tf.placeholder_with_default(0., shape=())
    }

    # Create model
    model = None
    if FLAGS.model == 'gcn_ae':
        # Standard Graph Autoencoder
        model = GCNModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'gcn_vae':
        # Standard Graph Variational Autoencoder
        model = GCNModelVAE(placeholders, num_features, num_nodes,
                            features_nonzero)
    elif FLAGS.model == 'source_target_gcn_ae':
        # Source-Target Graph Autoencoder
        if FLAGS.dimension % 2 != 0:
            raise ValueError('Dimension must be even for Source-Target models')
        model = SourceTargetGCNModelAE(placeholders, num_features,
                                     features_nonzero)
    elif FLAGS.model == 'source_target_gcn_vae':
        # Source-Target Graph Variational Autoencoder
        if FLAGS.dimension % 2 != 0:
            raise ValueError('Dimension must be even for Source-Target models')
        model = SourceTargetGCNModelVAE(placeholders, num_features,
                                      num_nodes, features_nonzero)
    elif FLAGS.model == 'gravity_gcn_ae':
        # Gravity-Inspired Graph Autoencoder
        model = GravityGCNModelAE(placeholders, num_features,
                                  features_nonzero)
    elif FLAGS.model == 'gravity_gcn_vae':
        # Gravity-Inspired Graph Variational Autoencoder
        model = GravityGCNModelVAE(placeholders, num_features, num_nodes,
                                   features_nonzero)
    else:
        raise ValueError('Undefined model!')

    # Optimizer (see tkipf/gae original GAE repository for details)
    pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
    norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0]
                                                - adj.sum()) * 2)
    with tf.name_scope('optimizer'):
        # Optimizer for Non-Variational Autoencoders
        if FLAGS.model in ('gcn_ae', 'source_target_gcn_ae', 'gravity_gcn_ae'):
            opt = OptimizerAE(preds = model.reconstructions,
                              labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                            validate_indices = False), [-1]),
                              pos_weight = pos_weight,
                              norm = norm)
        # Optimizer for Variational Autoencoders
        elif FLAGS.model in ('gcn_vae', 'source_target_gcn_vae', 'gravity_gcn_vae'):
            opt = OptimizerVAE(preds = model.reconstructions,
                               labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                             validate_indices = False), [-1]),
                               model = model,
                               num_nodes = num_nodes,
                               pos_weight = pos_weight,
                               norm = norm)

    # Normalization and preprocessing on adjacency matrix
    adj_norm = preprocess_graph(adj)
    adj_label = sparse_to_tuple(adj + sp.eye(adj.shape[0]))

    # Initialize TF session
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    # Model training
    if FLAGS.verbose:
        print("Training...")
    # Flag to compute total running time
    t_start = time.time()
    for epoch in range(FLAGS.epochs):
        # Flag to compute running time for each epoch
        t = time.time()
        # Construct feed dictionary
        feed_dict = construct_feed_dict(adj_norm, adj_label, features,
                                        placeholders)
        feed_dict.update({placeholders['dropout']: FLAGS.dropout})
        # Weight update
        outs = sess.run([opt.opt_op, opt.cost, opt.accuracy],
                        feed_dict = feed_dict)
        # Compute average loss
        avg_cost = outs[1]
        if FLAGS.verbose:
            # Display epoch information
            print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(avg_cost),
                  "time=", "{:.5f}".format(time.time() - t))
            # Validation (implemented for Task 1 only)
            if FLAGS.validation and FLAGS.task == 'task_1':
                feed_dict.update({placeholders['dropout']: 0})
                emb = sess.run(model.z_mean, feed_dict = feed_dict)
                feed_dict.update({placeholders['dropout']: FLAGS.dropout})
                val_roc, val_ap = compute_scores(val_edges, val_edges_false, emb)
                print("val_roc=", "{:.5f}".format(val_roc), "val_ap=", "{:.5f}".format(val_ap))

    # Compute total running time
    mean_time.append(time.time() - t_start)

    # Get embedding from model
    emb = sess.run(model.z_mean, feed_dict = feed_dict)

    # Test model
    if FLAGS.verbose:
        print("Testing model...")
    # Compute ROC and AP scores on test sets
    roc_score, ap_score = compute_scores(test_edges, test_edges_false, emb)
    # Append to list of scores over all runs
    mean_roc.append(roc_score)
    mean_ap.append(ap_score)

# Report final results
print("\nTest results for", FLAGS.model,
      "model on", FLAGS.dataset, "on", FLAGS.task, "\n",
      "___________________________________________________\n")

print("AUC scores\n", mean_roc)
print("Mean AUC score: ", np.mean(mean_roc),
      "\nStd of AUC scores: ", np.std(mean_roc), "\n \n")

print("AP scores \n", mean_ap)
print("Mean AP score: ", np.mean(mean_ap),
      "\nStd of AP scores: ", np.std(mean_ap), "\n \n")

print("Running times\n", mean_time)
print("Mean running time: ", np.mean(mean_time),
      "\nStd of running time: ", np.std(mean_time), "\n \n")

Loading data...
Masking test edges...
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 2.25637 time= 0.10298
Epoch: 0002 train_loss= 1.67327 time= 0.00300
Epoch: 0003 train_loss= 1.16032 time= 0.00396
Epoch: 0004 train_loss= 1.07134 time= 0.00295
Epoch: 0005 train_loss= 1.85339 time= 0.00402
Epoch: 0006 train_loss= 1.05173 time= 0.00302
Epoch: 0007 train_loss= 1.05395 time= 0.00295
Epoch: 0008 train_loss= 1.04044 time= 0.00301
Epoch: 0009 train_loss= 1.08007 time= 0.00399
Epoch: 0010 train_loss= 0.96974 time= 0.00408
Epoch: 0011 train_loss= 0.97528 time= 0.00292
Epoch: 0012 train_loss= 0.96696 time= 0.00196
Epoch: 0013 train_loss= 0.86986 time= 0.00304
Epoch: 0014 train_loss= 0.87427 time= 0.00393
Epoch: 0015 train_loss= 0.84121 time= 0.00400
Epoch: 0016 train_loss= 0.76942 time= 0.00403
Epoch: 0017 train_loss= 0.78720 time= 0.00397
Epoch: 0018 train_loss= 0.77885 time= 0.00400
Epoch: 0019 train_loss= 0.79574 time= 0.00300
Epoch: 0020 train_loss= 0.75690 time= 0.00

Epoch: 0181 train_loss= 0.29254 time= 0.00304
Epoch: 0182 train_loss= 0.28524 time= 0.00404
Epoch: 0183 train_loss= 0.30024 time= 0.00392
Epoch: 0184 train_loss= 0.28458 time= 0.00393
Epoch: 0185 train_loss= 0.29206 time= 0.00300
Epoch: 0186 train_loss= 0.30396 time= 0.00301
Epoch: 0187 train_loss= 0.29267 time= 0.00399
Epoch: 0188 train_loss= 0.30251 time= 0.00302
Epoch: 0189 train_loss= 0.29265 time= 0.00299
Epoch: 0190 train_loss= 0.29386 time= 0.00300
Epoch: 0191 train_loss= 0.30272 time= 0.00300
Epoch: 0192 train_loss= 0.29128 time= 0.00200
Epoch: 0193 train_loss= 0.29871 time= 0.00294
Epoch: 0194 train_loss= 0.28918 time= 0.00300
Epoch: 0195 train_loss= 0.29137 time= 0.00200
Epoch: 0196 train_loss= 0.28999 time= 0.00200
Epoch: 0197 train_loss= 0.29455 time= 0.00300
Epoch: 0198 train_loss= 0.28869 time= 0.00304
Epoch: 0199 train_loss= 0.29494 time= 0.00299
Epoch: 0200 train_loss= 0.29408 time= 0.00300
Testing model...
Masking test edges...
Preprocessing and Initializing...
Trainin

Epoch: 0188 train_loss= 0.30262 time= 0.00301
Epoch: 0189 train_loss= 0.28918 time= 0.00300
Epoch: 0190 train_loss= 0.29478 time= 0.00199
Epoch: 0191 train_loss= 0.29665 time= 0.00295
Epoch: 0192 train_loss= 0.28789 time= 0.00301
Epoch: 0193 train_loss= 0.29962 time= 0.00399
Epoch: 0194 train_loss= 0.28861 time= 0.00306
Epoch: 0195 train_loss= 0.30940 time= 0.00302
Epoch: 0196 train_loss= 0.29907 time= 0.00298
Epoch: 0197 train_loss= 0.29650 time= 0.00401
Epoch: 0198 train_loss= 0.29981 time= 0.00500
Epoch: 0199 train_loss= 0.29553 time= 0.00399
Epoch: 0200 train_loss= 0.30144 time= 0.00410
Testing model...
Masking test edges...
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 2.33324 time= 0.08601
Epoch: 0002 train_loss= 1.70527 time= 0.00701
Epoch: 0003 train_loss= 1.24406 time= 0.00301
Epoch: 0004 train_loss= 1.14905 time= 0.00300
Epoch: 0005 train_loss= 1.32582 time= 0.00301
Epoch: 0006 train_loss= 1.07754 time= 0.00399
Epoch: 0007 train_loss= 1.11292 time= 0.0

Epoch: 0178 train_loss= 0.30182 time= 0.00295
Epoch: 0179 train_loss= 0.30743 time= 0.00400
Epoch: 0180 train_loss= 0.30340 time= 0.00300
Epoch: 0181 train_loss= 0.31246 time= 0.00308
Epoch: 0182 train_loss= 0.31175 time= 0.00399
Epoch: 0183 train_loss= 0.29747 time= 0.00297
Epoch: 0184 train_loss= 0.30144 time= 0.00314
Epoch: 0185 train_loss= 0.30664 time= 0.00293
Epoch: 0186 train_loss= 0.30628 time= 0.00400
Epoch: 0187 train_loss= 0.30311 time= 0.00317
Epoch: 0188 train_loss= 0.30229 time= 0.00285
Epoch: 0189 train_loss= 0.30485 time= 0.00299
Epoch: 0190 train_loss= 0.29570 time= 0.00395
Epoch: 0191 train_loss= 0.29755 time= 0.00296
Epoch: 0192 train_loss= 0.29995 time= 0.00304
Epoch: 0193 train_loss= 0.31178 time= 0.00395
Epoch: 0194 train_loss= 0.31221 time= 0.00401
Epoch: 0195 train_loss= 0.30012 time= 0.00399
Epoch: 0196 train_loss= 0.29332 time= 0.00305
Epoch: 0197 train_loss= 0.29617 time= 0.00317
Epoch: 0198 train_loss= 0.30058 time= 0.00279
Epoch: 0199 train_loss= 0.29433 ti

Epoch: 0185 train_loss= 0.29463 time= 0.00300
Epoch: 0186 train_loss= 0.30266 time= 0.00302
Epoch: 0187 train_loss= 0.29707 time= 0.00296
Epoch: 0188 train_loss= 0.29655 time= 0.00300
Epoch: 0189 train_loss= 0.29206 time= 0.00500
Epoch: 0190 train_loss= 0.28747 time= 0.00500
Epoch: 0191 train_loss= 0.29835 time= 0.00308
Epoch: 0192 train_loss= 0.29916 time= 0.00295
Epoch: 0193 train_loss= 0.29235 time= 0.00306
Epoch: 0194 train_loss= 0.28878 time= 0.00197
Epoch: 0195 train_loss= 0.29704 time= 0.00307
Epoch: 0196 train_loss= 0.28714 time= 0.00394
Epoch: 0197 train_loss= 0.29880 time= 0.00313
Epoch: 0198 train_loss= 0.28987 time= 0.00386
Epoch: 0199 train_loss= 0.29570 time= 0.00300
Epoch: 0200 train_loss= 0.29963 time= 0.00306
Testing model...
Masking test edges...
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 2.24127 time= 0.11800
Epoch: 0002 train_loss= 1.63508 time= 0.00504
Epoch: 0003 train_loss= 1.16763 time= 0.00295
Epoch: 0004 train_loss= 1.13464 time= 0.0

...

Epoch: 0169 train_loss= 0.31426 time= 0.00299
Epoch: 0170 train_loss= 0.32091 time= 0.00397
Epoch: 0171 train_loss= 0.33267 time= 0.00299
Epoch: 0172 train_loss= 0.31200 time= 0.00300
Epoch: 0173 train_loss= 0.31647 time= 0.00201
Epoch: 0174 train_loss= 0.31124 time= 0.00399
Epoch: 0175 train_loss= 0.31349 time= 0.00405
Epoch: 0176 train_loss= 0.31114 time= 0.00398
Epoch: 0177 train_loss= 0.31660 time= 0.00298
Epoch: 0178 train_loss= 0.30633 time= 0.00306
Epoch: 0179 train_loss= 0.32285 time= 0.00304
Epoch: 0180 train_loss= 0.30318 time= 0.00302
Epoch: 0181 train_loss= 0.31737 time= 0.00296
Epoch: 0182 train_loss= 0.31439 time= 0.00296
Epoch: 0183 train_loss= 0.30732 time= 0.00301
Epoch: 0184 train_loss= 0.30233 time= 0.00300
Epoch: 0185 train_loss= 0.31011 time= 0.00305
Epoch: 0186 train_loss= 0.29907 time= 0.00301
Epoch: 0187 train_loss= 0.31021 time= 0.00296
Epoch: 0188 train_loss= 0.31307 time= 0.00399
Epoch: 0189 train_loss= 0.30766 time= 0.00407
Epoch: 0190 train_loss= 0.32216 ti

Epoch: 0166 train_loss= 0.31698 time= 0.00407
Epoch: 0167 train_loss= 0.30578 time= 0.00397
Epoch: 0168 train_loss= 0.30147 time= 0.00296
Epoch: 0169 train_loss= 0.30783 time= 0.00305
Epoch: 0170 train_loss= 0.29686 time= 0.00296
Epoch: 0171 train_loss= 0.30887 time= 0.00309
Epoch: 0172 train_loss= 0.30721 time= 0.00391
Epoch: 0173 train_loss= 0.30581 time= 0.00287
Epoch: 0174 train_loss= 0.30673 time= 0.00401
Epoch: 0175 train_loss= 0.29805 time= 0.00305
Epoch: 0176 train_loss= 0.32256 time= 0.00302
Epoch: 0177 train_loss= 0.29175 time= 0.00309
Epoch: 0178 train_loss= 0.31083 time= 0.00290
Epoch: 0179 train_loss= 0.30729 time= 0.00301
Epoch: 0180 train_loss= 0.30256 time= 0.00308
Epoch: 0181 train_loss= 0.30166 time= 0.00295
Epoch: 0182 train_loss= 0.29957 time= 0.00313
Epoch: 0183 train_loss= 0.30396 time= 0.00296
Epoch: 0184 train_loss= 0.30758 time= 0.00399
Epoch: 0185 train_loss= 0.29765 time= 0.00303
Epoch: 0186 train_loss= 0.30479 time= 0.00294
Epoch: 0187 train_loss= 0.29700 ti

Epoch: 0150 train_loss= 0.32591 time= 0.00405
Epoch: 0151 train_loss= 0.30988 time= 0.00573
Epoch: 0152 train_loss= 0.30829 time= 0.00400
Epoch: 0153 train_loss= 0.31705 time= 0.00308
Epoch: 0154 train_loss= 0.30191 time= 0.00297
Epoch: 0155 train_loss= 0.29255 time= 0.00386
Epoch: 0156 train_loss= 0.31189 time= 0.00392
Epoch: 0157 train_loss= 0.29907 time= 0.00406
Epoch: 0158 train_loss= 0.30690 time= 0.00495
Epoch: 0159 train_loss= 0.31392 time= 0.00265
Epoch: 0160 train_loss= 0.31185 time= 0.00394
Epoch: 0161 train_loss= 0.31203 time= 0.00401
Epoch: 0162 train_loss= 0.30297 time= 0.00518
Epoch: 0163 train_loss= 0.32358 time= 0.00396
Epoch: 0164 train_loss= 0.30800 time= 0.00404
Epoch: 0165 train_loss= 0.31580 time= 0.00495
Epoch: 0166 train_loss= 0.30429 time= 0.00495
Epoch: 0167 train_loss= 0.29911 time= 0.00403
Epoch: 0168 train_loss= 0.31304 time= 0.00402
Epoch: 0169 train_loss= 0.31258 time= 0.00490
Epoch: 0170 train_loss= 0.30377 time= 0.00504
Epoch: 0171 train_loss= 0.29671 ti

Epoch: 0160 train_loss= 0.29568 time= 0.00394
Epoch: 0161 train_loss= 0.30182 time= 0.00299
Epoch: 0162 train_loss= 0.30360 time= 0.00301
Epoch: 0163 train_loss= 0.29165 time= 0.00405
Epoch: 0164 train_loss= 0.28952 time= 0.00291
Epoch: 0165 train_loss= 0.29854 time= 0.00295
Epoch: 0166 train_loss= 0.29495 time= 0.00300
Epoch: 0167 train_loss= 0.29353 time= 0.00299
Epoch: 0168 train_loss= 0.28828 time= 0.00297
Epoch: 0169 train_loss= 0.31275 time= 0.00505
Epoch: 0170 train_loss= 0.29590 time= 0.00303
Epoch: 0171 train_loss= 0.28698 time= 0.00306
Epoch: 0172 train_loss= 0.29020 time= 0.00402
Epoch: 0173 train_loss= 0.29595 time= 0.00493
Epoch: 0174 train_loss= 0.29861 time= 0.00304
Epoch: 0175 train_loss= 0.29217 time= 0.00302
Epoch: 0176 train_loss= 0.29063 time= 0.00399
Epoch: 0177 train_loss= 0.28725 time= 0.00400
Epoch: 0178 train_loss= 0.29678 time= 0.00403
Epoch: 0179 train_loss= 0.29167 time= 0.00391
Epoch: 0180 train_loss= 0.29778 time= 0.00301
Epoch: 0181 train_loss= 0.29515 ti

Epoch: 0149 train_loss= 0.30424 time= 0.00500
Epoch: 0150 train_loss= 0.29719 time= 0.00400
Epoch: 0151 train_loss= 0.29168 time= 0.00383
Epoch: 0152 train_loss= 0.29670 time= 0.00400
Epoch: 0153 train_loss= 0.29380 time= 0.00325
Epoch: 0154 train_loss= 0.29993 time= 0.00376
Epoch: 0155 train_loss= 0.28829 time= 0.00400
Epoch: 0156 train_loss= 0.29788 time= 0.00304
Epoch: 0157 train_loss= 0.29255 time= 0.00397
Epoch: 0158 train_loss= 0.29675 time= 0.00400
Epoch: 0159 train_loss= 0.29470 time= 0.00405
Epoch: 0160 train_loss= 0.29866 time= 0.00293
Epoch: 0161 train_loss= 0.28304 time= 0.00401
Epoch: 0162 train_loss= 0.28976 time= 0.00400
Epoch: 0163 train_loss= 0.29856 time= 0.00499
Epoch: 0164 train_loss= 0.28605 time= 0.00292
Epoch: 0165 train_loss= 0.28523 time= 0.00299
Epoch: 0166 train_loss= 0.30490 time= 0.00296
Epoch: 0167 train_loss= 0.29008 time= 0.00405
Epoch: 0168 train_loss= 0.29053 time= 0.00396
Epoch: 0169 train_loss= 0.29141 time= 0.00394
Epoch: 0170 train_loss= 0.29641 ti